## FINGER TAPPING ANALYSIS
##### Data collected from patients with neurodegenerative disorders as well as healthy controls

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import time
import sys
from tqdm import tqdm
import os
import pandas as pd
import seaborn as sns
sns.set(style="darkgrid")
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
import math
from statsmodels.sandbox.stats.multicomp import multipletests
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras
from keras.models import Model, model_from_yaml
from keras.constraints import max_norm
from keras.layers import Input, Conv1D, Flatten, Dropout, MaxPooling1D, Dense
from keras.layers import Activation, BatchNormalization, concatenate
from keras import optimizers
from keras.initializers import he_normal
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
%matplotlib inline

# from IPython.display import Audio
# sound_file = './sound/beep.wav'

C:\Users\Vladislava\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Read the data

train = scipy.io.loadmat('TrainData.mat')
val = scipy.io.loadmat('ValData.mat')


In [3]:
Xtrain, Ytrain = train['X'], train['Y']
Xval, Yval = val['X'], val['Y']

### Arite, lez make a model

In [4]:
def CNNModel(inputShape, nConvLayers, kernel_size):
    
    input1 = Input(shape = inputShape)

    #convolutions
    x = input1
    
    
    for i in range(0,nConvLayers):
        
        nFilters = 128 if i>1 else 32*(i+1)
        inside = 3 if i>1 else 2
        for temp in range(0,inside):
            x = Conv1D(filters = nFilters,
                  kernel_size = kernel_size,
                  padding = 'same',
                  strides = 1,
                  kernel_initializer = 'he_normal',
                  name = 'Conv1x5{}{}'.format(i,temp))(x)
            
            x = Activation('relu',name='ReLu{}{}'.format(i,temp))(x)
            x = BatchNormalization()(x)
            
        x = MaxPooling1D(2,
                      padding = 'same',
                      strides = 1,
                      name = 'MaxPooling1D{}'.format(i))(x)
    
    x = Dropout(0.5)(x)
    
    # Fully connected
    x = Flatten()(x)
    
    x = Dense(128,
              kernel_constraint = max_norm(3),
              kernel_initializer = 'he_normal')(x)
    x = Activation('relu', name = 'reLU_dense')(x)
    x = Dropout(0.6)(x)
    
    x = Dense(4)(x)
    x = Activation('softmax',name = 'Softmax')(x)

    m = Model(input1,x)
    return m


In [5]:
def saveModelTopology(model,modelName):
    model_json = model.to_yaml()
    with open(modelName+'.yaml', "w") as yaml_file:
        yaml_file.write(model_json)
    print("Saved model to {}.yaml".format(modelName))
    return


In [6]:
def defCallbacks(weightFile):
    
    checkpoint = ModelCheckpoint(weightFile,
                                 monitor='val_acc',
                                 verbose=1,
                                 save_best_only = True,
                                 save_weights_only = True,
                                 mode='max')
    early = EarlyStopping(monitor='val_acc',
                          patience = 17,
                          verbose = 1,
                          mode='max')
#     def step_decay(epoch):
#         initial_lrate = 0.001
#         rate_drop = 0.25
#         nEpochs = 5
#         lrate = initial_lrate * math.pow(rate_drop, math.floor(epoch/nEpochs))
#         return lrate 
    
#     lrate = LearningRateScheduler(step_decay, verbose = 1)
    lr_plateau = ReduceLROnPlateau(monitor = 'val_acc', factor = 0.2,
                                  patience = 6, min_lr = 0.0000000001,
                                  verbose = 1)
    
    return [checkpoint, early, lr_plateau]
    

In [7]:
def fitModel(model, modelName, Xtrain, Ytrain, Xval, Yval, epochs,batch_size):
    
    tic = time.time()
    
    # make file name
    tm = time.gmtime()
    weightFile = 'BEST_WEIGHTS{}.{}.{}.{}.{}.{}.h5'.format(modelName,tm[2],tm[1],tm[0],tm[3]+1,tm[4])
    
    #define callbacks
    callbacks = defCallbacks(weightFile)
    
    # FIT THE MODEL
    history = model.fit(x = Xtrain, y = Ytrain,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data = (Xval,Yval),
                        callbacks = callbacks)
    toc = time.time()
    print("Finished training in {} min ({} h)".format(round((toc-tic)/60,2),round((toc-tic)/3600,2)))

    
    # Save the weights
    #model.save_weights(str(modelName)+'.h5') # ???????
    
    return history

In [8]:
# def evaluateModel(model, modelName, Xval, Yval):
#     tic = time.time()
#     predictions = model.predict(Xval)
#     toc = time.time()
#     print('Finished prediction in: {} min'.format(round((toc-tic)/60,2)))

#     print('Evaluating...')
#     score = model.evaluate(Xval,Yval,verbose=1)
#     print(score)

#     #Save that stuff too pls.
#     tm = time.gmtime()    
#     predictionFile = 'Predictions-{}.{}.{}.{}.{}.{}.csv'.format(modelName,tm[2],tm[1],tm[0],tm[3]+1,tm[4])

#     dfPredicted = pd.DataFrame(predictions)
#     dfPredicted = dfPredicted.idxmax(axis =1)
#     dfExpected = pd.DataFrame(Yval)
#     dfExpected = dfExpected.idxmax(axis =1)
#     df = pd.DataFrame({'Predicted':dfPredicted, 'Expected':dfExpected})
#     df.to_csv(predictionFile,index = False)
#     print('Saved predictions to: ', predictionFile)
    
#     bingos = sum(df['Predicted'] ==df['Expected'])
#     accRly = 100*bingos/df.shape[0]
#     print('Currently your actual accuracy on Xval is: {}%'.format(round(accRly,2)))
    
#     return accRly

In [9]:
histories = []
#modelDepths = []
accuracies = []
nConvLayers = 5   
epochs = 200
batch_size = 16
kernelSizes = [9]

for kernel_size in kernelSizes:
    for i in range(4):
    
        model = CNNModel((Xtrain.shape[1],Xtrain.shape[2]),nConvLayers, kernel_size)
        opt = optimizers.SGD(lr=0.0001, momentum=0.9, nesterov=True)
        model.compile(optimizer = opt,
                 loss='categorical_crossentropy',
                 metrics = ['accuracy'])
        #model.summary()

        modelName = 'CNN'+str(batch_size)+'Batch'+ str(nConvLayers)+'KERNEL'+str(kernel_size)
        #saveModelTopology(model,modelName)
        model.save(modelName+'CEO.h5')

        print('TRAINING...')

        history = fitModel(model,modelName, Xtrain, Ytrain, Xval, Yval, epochs,batch_size)
        histories.append(history.history)   
        #modelDepths.append(nConvLayers)
        
        



    
#     acc = evaluateModel(model, modelName, Xval, Yval)
#     accuracies.append(acc)
    
    
#     with open("hist.txt", "w") as f:
#         for h in histories:
#             f.write(str(h) +"\n")

#     with open("hist.txt", "r") as f:
#         for line in f:
#             histo.append(int(line.strip()))

TRAINING...
Train on 2719 samples, validate on 375 samples
Epoch 1/200
2719/2719 [==============================] - ETA: 13:38 - loss: 4.9834 - acc: 0.25 - ETA: 7:11 - loss: 4.4661 - acc: 0.2500 - ETA: 5:00 - loss: 3.9202 - acc: 0.250 - ETA: 3:55 - loss: 3.9338 - acc: 0.203 - ETA: 3:16 - loss: 3.8457 - acc: 0.212 - ETA: 2:50 - loss: 3.7023 - acc: 0.229 - ETA: 2:31 - loss: 3.5497 - acc: 0.241 - ETA: 2:16 - loss: 3.4317 - acc: 0.257 - ETA: 2:05 - loss: 3.5136 - acc: 0.256 - ETA: 1:57 - loss: 3.4936 - acc: 0.250 - ETA: 1:49 - loss: 3.4721 - acc: 0.255 - ETA: 1:43 - loss: 3.4025 - acc: 0.255 - ETA: 1:38 - loss: 3.3250 - acc: 0.259 - ETA: 1:33 - loss: 3.3510 - acc: 0.250 - ETA: 1:29 - loss: 3.3458 - acc: 0.262 - ETA: 1:26 - loss: 3.4370 - acc: 0.257 - ETA: 1:23 - loss: 3.4594 - acc: 0.253 - ETA: 1:20 - loss: 3.4059 - acc: 0.250 - ETA: 1:17 - loss: 3.3847 - acc: 0.250 - ETA: 1:15 - loss: 3.3883 - acc: 0.253 - ETA: 1:13 - loss: 3.3731 - acc: 0.261 - ETA: 1:11 - loss: 3.3320 - acc: 0.261 - ETA

2719/2719 [==============================] - ETA: 50s - loss: 1.3410 - acc: 0.37 - ETA: 48s - loss: 1.2641 - acc: 0.46 - ETA: 47s - loss: 1.3259 - acc: 0.39 - ETA: 46s - loss: 1.2505 - acc: 0.45 - ETA: 46s - loss: 1.3220 - acc: 0.40 - ETA: 45s - loss: 1.3259 - acc: 0.38 - ETA: 45s - loss: 1.3174 - acc: 0.38 - ETA: 45s - loss: 1.3023 - acc: 0.39 - ETA: 44s - loss: 1.2900 - acc: 0.40 - ETA: 44s - loss: 1.2654 - acc: 0.41 - ETA: 44s - loss: 1.2460 - acc: 0.42 - ETA: 43s - loss: 1.2346 - acc: 0.42 - ETA: 43s - loss: 1.2340 - acc: 0.42 - ETA: 43s - loss: 1.2250 - acc: 0.43 - ETA: 42s - loss: 1.2243 - acc: 0.42 - ETA: 42s - loss: 1.2246 - acc: 0.41 - ETA: 42s - loss: 1.2255 - acc: 0.41 - ETA: 41s - loss: 1.2246 - acc: 0.41 - ETA: 41s - loss: 1.2170 - acc: 0.41 - ETA: 41s - loss: 1.2174 - acc: 0.41 - ETA: 40s - loss: 1.2217 - acc: 0.41 - ETA: 40s - loss: 1.2142 - acc: 0.42 - ETA: 40s - loss: 1.2136 - acc: 0.42 - ETA: 40s - loss: 1.2097 - acc: 0.42 - ETA: 39s - loss: 1.2106 - acc: 0.42 - ETA: 

2719/2719 [==============================] - ETA: 50s - loss: 0.9922 - acc: 0.50 - ETA: 47s - loss: 0.9484 - acc: 0.56 - ETA: 47s - loss: 0.9633 - acc: 0.58 - ETA: 46s - loss: 0.9961 - acc: 0.57 - ETA: 45s - loss: 1.0260 - acc: 0.55 - ETA: 45s - loss: 1.0466 - acc: 0.51 - ETA: 45s - loss: 1.0485 - acc: 0.50 - ETA: 44s - loss: 1.0345 - acc: 0.50 - ETA: 44s - loss: 1.0105 - acc: 0.52 - ETA: 44s - loss: 0.9849 - acc: 0.53 - ETA: 43s - loss: 0.9995 - acc: 0.52 - ETA: 43s - loss: 0.9949 - acc: 0.52 - ETA: 43s - loss: 0.9988 - acc: 0.51 - ETA: 42s - loss: 1.0276 - acc: 0.50 - ETA: 42s - loss: 1.0407 - acc: 0.48 - ETA: 42s - loss: 1.0428 - acc: 0.48 - ETA: 42s - loss: 1.0343 - acc: 0.50 - ETA: 41s - loss: 1.0310 - acc: 0.51 - ETA: 41s - loss: 1.0236 - acc: 0.52 - ETA: 41s - loss: 1.0287 - acc: 0.52 - ETA: 40s - loss: 1.0303 - acc: 0.52 - ETA: 40s - loss: 1.0317 - acc: 0.52 - ETA: 40s - loss: 1.0323 - acc: 0.52 - ETA: 39s - loss: 1.0245 - acc: 0.53 - ETA: 39s - loss: 1.0336 - acc: 0.53 - ETA: 

2719/2719 [==============================] - ETA: 52s - loss: 0.8724 - acc: 0.56 - ETA: 49s - loss: 0.8224 - acc: 0.62 - ETA: 48s - loss: 0.8056 - acc: 0.64 - ETA: 47s - loss: 0.8134 - acc: 0.64 - ETA: 47s - loss: 0.8721 - acc: 0.60 - ETA: 46s - loss: 0.8953 - acc: 0.60 - ETA: 45s - loss: 0.8390 - acc: 0.63 - ETA: 45s - loss: 0.8475 - acc: 0.61 - ETA: 44s - loss: 0.8552 - acc: 0.59 - ETA: 44s - loss: 0.8664 - acc: 0.58 - ETA: 44s - loss: 0.8696 - acc: 0.59 - ETA: 43s - loss: 0.8492 - acc: 0.60 - ETA: 43s - loss: 0.8719 - acc: 0.58 - ETA: 43s - loss: 0.8827 - acc: 0.58 - ETA: 42s - loss: 0.9114 - acc: 0.56 - ETA: 42s - loss: 0.9125 - acc: 0.56 - ETA: 42s - loss: 0.9070 - acc: 0.56 - ETA: 42s - loss: 0.9142 - acc: 0.55 - ETA: 41s - loss: 0.9097 - acc: 0.55 - ETA: 41s - loss: 0.9163 - acc: 0.55 - ETA: 41s - loss: 0.9044 - acc: 0.57 - ETA: 40s - loss: 0.9045 - acc: 0.57 - ETA: 40s - loss: 0.9103 - acc: 0.58 - ETA: 40s - loss: 0.9077 - acc: 0.58 - ETA: 39s - loss: 0.9055 - acc: 0.58 - ETA: 

2719/2719 [==============================] - ETA: 51s - loss: 0.4119 - acc: 0.81 - ETA: 49s - loss: 0.6782 - acc: 0.68 - ETA: 47s - loss: 0.6731 - acc: 0.70 - ETA: 47s - loss: 0.6322 - acc: 0.75 - ETA: 46s - loss: 0.6469 - acc: 0.73 - ETA: 46s - loss: 0.6576 - acc: 0.75 - ETA: 45s - loss: 0.6557 - acc: 0.75 - ETA: 45s - loss: 0.6556 - acc: 0.75 - ETA: 44s - loss: 0.6691 - acc: 0.75 - ETA: 44s - loss: 0.6649 - acc: 0.75 - ETA: 44s - loss: 0.6521 - acc: 0.74 - ETA: 43s - loss: 0.6653 - acc: 0.72 - ETA: 43s - loss: 0.6461 - acc: 0.73 - ETA: 43s - loss: 0.6495 - acc: 0.73 - ETA: 42s - loss: 0.6759 - acc: 0.71 - ETA: 42s - loss: 0.6768 - acc: 0.70 - ETA: 42s - loss: 0.6798 - acc: 0.69 - ETA: 42s - loss: 0.6686 - acc: 0.70 - ETA: 41s - loss: 0.6689 - acc: 0.70 - ETA: 41s - loss: 0.6821 - acc: 0.69 - ETA: 41s - loss: 0.6858 - acc: 0.69 - ETA: 40s - loss: 0.6880 - acc: 0.68 - ETA: 40s - loss: 0.6926 - acc: 0.68 - ETA: 40s - loss: 0.6855 - acc: 0.69 - ETA: 40s - loss: 0.6896 - acc: 0.69 - ETA: 

...

2719/2719 [==============================] - ETA: 45s - loss: 0.0041 - acc: 1.00 - ETA: 45s - loss: 0.0027 - acc: 1.00 - ETA: 45s - loss: 0.0062 - acc: 1.00 - ETA: 45s - loss: 0.0182 - acc: 1.00 - ETA: 45s - loss: 0.0164 - acc: 1.00 - ETA: 44s - loss: 0.0173 - acc: 1.00 - ETA: 44s - loss: 0.0163 - acc: 1.00 - ETA: 44s - loss: 0.0154 - acc: 1.00 - ETA: 44s - loss: 0.0138 - acc: 1.00 - ETA: 43s - loss: 0.0126 - acc: 1.00 - ETA: 43s - loss: 0.0115 - acc: 1.00 - ETA: 43s - loss: 0.0109 - acc: 1.00 - ETA: 43s - loss: 0.0101 - acc: 1.00 - ETA: 42s - loss: 0.0095 - acc: 1.00 - ETA: 42s - loss: 0.0296 - acc: 0.99 - ETA: 42s - loss: 0.0278 - acc: 0.99 - ETA: 42s - loss: 0.0262 - acc: 0.99 - ETA: 41s - loss: 0.0253 - acc: 0.99 - ETA: 41s - loss: 0.0268 - acc: 0.99 - ETA: 41s - loss: 0.0350 - acc: 0.99 - ETA: 40s - loss: 0.0333 - acc: 0.99 - ETA: 40s - loss: 0.0318 - acc: 0.99 - ETA: 40s - loss: 0.0306 - acc: 0.99 - ETA: 40s - loss: 0.0294 - acc: 0.99 - ETA: 39s - loss: 0.0286 - acc: 0.99 - ETA: 

2719/2719 [==============================] - ETA: 47s - loss: 0.0015 - acc: 1.00 - ETA: 46s - loss: 8.2667e-04 - acc: 1.00 - ETA: 45s - loss: 0.0043 - acc: 1.0000   - ETA: 45s - loss: 0.0080 - acc: 1.00 - ETA: 45s - loss: 0.0066 - acc: 1.00 - ETA: 45s - loss: 0.0085 - acc: 1.00 - ETA: 44s - loss: 0.0074 - acc: 1.00 - ETA: 44s - loss: 0.0067 - acc: 1.00 - ETA: 44s - loss: 0.0060 - acc: 1.00 - ETA: 43s - loss: 0.0176 - acc: 0.98 - ETA: 43s - loss: 0.0163 - acc: 0.98 - ETA: 43s - loss: 0.0173 - acc: 0.98 - ETA: 43s - loss: 0.0169 - acc: 0.99 - ETA: 43s - loss: 0.0158 - acc: 0.99 - ETA: 42s - loss: 0.0218 - acc: 0.98 - ETA: 42s - loss: 0.0206 - acc: 0.98 - ETA: 42s - loss: 0.0197 - acc: 0.98 - ETA: 41s - loss: 0.0186 - acc: 0.98 - ETA: 41s - loss: 0.0186 - acc: 0.99 - ETA: 41s - loss: 0.0208 - acc: 0.98 - ETA: 41s - loss: 0.0198 - acc: 0.98 - ETA: 40s - loss: 0.0189 - acc: 0.98 - ETA: 40s - loss: 0.0194 - acc: 0.98 - ETA: 40s - loss: 0.0186 - acc: 0.98 - ETA: 39s - loss: 0.0179 - acc: 0.99

2719/2719 [==============================] - ETA: 45s - loss: 0.0080 - acc: 1.00 - ETA: 45s - loss: 0.0090 - acc: 1.00 - ETA: 45s - loss: 0.0100 - acc: 1.00 - ETA: 45s - loss: 0.0931 - acc: 0.98 - ETA: 45s - loss: 0.0752 - acc: 0.98 - ETA: 44s - loss: 0.0632 - acc: 0.98 - ETA: 44s - loss: 0.0854 - acc: 0.97 - ETA: 44s - loss: 0.0748 - acc: 0.97 - ETA: 44s - loss: 0.0712 - acc: 0.97 - ETA: 44s - loss: 0.0645 - acc: 0.98 - ETA: 43s - loss: 0.0702 - acc: 0.97 - ETA: 43s - loss: 0.0643 - acc: 0.97 - ETA: 43s - loss: 0.0594 - acc: 0.98 - ETA: 42s - loss: 0.0554 - acc: 0.98 - ETA: 42s - loss: 0.0517 - acc: 0.98 - ETA: 42s - loss: 0.0491 - acc: 0.98 - ETA: 42s - loss: 0.0463 - acc: 0.98 - ETA: 41s - loss: 0.0442 - acc: 0.98 - ETA: 41s - loss: 0.0421 - acc: 0.98 - ETA: 41s - loss: 0.0401 - acc: 0.98 - ETA: 40s - loss: 0.0387 - acc: 0.98 - ETA: 40s - loss: 0.0382 - acc: 0.98 - ETA: 40s - loss: 0.0368 - acc: 0.98 - ETA: 40s - loss: 0.0354 - acc: 0.98 - ETA: 39s - loss: 0.0340 - acc: 0.99 - ETA: 

1488/2719 [===============>..............] - ETA: 45s - loss: 5.2699e-04 - acc: 1.00 - ETA: 45s - loss: 0.0044 - acc: 1.0000   - ETA: 45s - loss: 0.0036 - acc: 1.00 - ETA: 45s - loss: 0.0072 - acc: 1.00 - ETA: 45s - loss: 0.0061 - acc: 1.00 - ETA: 45s - loss: 0.0073 - acc: 1.00 - ETA: 44s - loss: 0.0063 - acc: 1.00 - ETA: 44s - loss: 0.0056 - acc: 1.00 - ETA: 44s - loss: 0.0511 - acc: 0.99 - ETA: 43s - loss: 0.0463 - acc: 0.99 - ETA: 43s - loss: 0.0421 - acc: 0.99 - ETA: 43s - loss: 0.0387 - acc: 0.99 - ETA: 43s - loss: 0.0357 - acc: 0.99 - ETA: 42s - loss: 0.0334 - acc: 0.99 - ETA: 42s - loss: 0.0314 - acc: 0.99 - ETA: 42s - loss: 0.0297 - acc: 0.99 - ETA: 42s - loss: 0.0280 - acc: 0.99 - ETA: 41s - loss: 0.0269 - acc: 0.99 - ETA: 41s - loss: 0.0255 - acc: 0.99 - ETA: 41s - loss: 0.0270 - acc: 0.99 - ETA: 41s - loss: 0.0258 - acc: 0.99 - ETA: 40s - loss: 0.0247 - acc: 0.99 - ETA: 40s - loss: 0.0237 - acc: 0.99 - ETA: 40s - loss: 0.0247 - acc: 0.99 - ETA: 39s - loss: 0.0273 - acc: 0.99

KeyboardInterrupt: 

In [ ]:
def checkOutHistory(history):
    

    # plot Accuracy over Epochs
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train Acc','Val Acc'])
    plt.title('Accuracy for {} over epochs'.format(modelName))
    plt.show()


    # plot Loss over Epochs
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train Loss','Val Loss'])
    plt.title('Loss for {} over epochs'.format(modelName))
    plt.show()
    
    print("Max val accuracy: ", max(history['val_acc']))
    print("Max train accuracy: ", max(history['acc']))
        
    return


In [ ]:
for h in histories:
    checkOutHistory(h)

In [ ]:

with open("hist9kernel.txt", "w") as f:
    for h in histories:
        f.write(str(h) +"\n")

model.summary()